In [1]:
import pickle
import math
import numpy as np
from param import Parameters
from components import *
# from initiate import add_vehicles_from_data, add_requests_from_data,add_requests_from_data2
from initiate import add_vehicles_from_data, add_requests_from_data
from opt_ridesharing import *

/Users/jiahuisun/anaconda3/envs/carpool-simu/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)
/Users/jiahuisun/anaconda3/envs/carpool-simu/lib/python3.7/site-packages/sklearn/base.py:306: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.19.1 when using version 0.21.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/Users/jiahuisun/anaconda3/envs/carpool-simu/lib/python3.7/site-packages/sklearn/base.py:306: UserWarning: Trying to unpickle estimator LogisticRegression from version 0.19.1 when using version 0.21.2. This might lead 

In [2]:
def init_params():
    '''
    init the params.
    :return:
    '''
    Parameters.Requests = []
    Parameters.RequestIndex = {}
    Parameters.Vehicles = []
    Parameters.RequestId = 0
    Parameters.VehicleId = 0
    Parameters.Trips = {}
    Parameters.RequestDone = []
    Parameters.computation_time = 0
    Parameters.IdleVehicles = []
    Parameters.unassigned_requests = []
    Parameters.VehiclePosition = {}
    Parameters.fea_set = {}
    Parameters.fea_set_last_iteration = {}
    Parameters.current_request_id = 0

In [3]:
Demand = pickle.load(open("./expdata1/demand20160108","rb"))
print(len(Demand))

383118


In [4]:
init_params()
Parameters.uber_percent = 0
Parameters.uber_vehicles = 0
Parameters.uberpool_vehicles = 3000
Parameters.uber_request_percent = 0
Parameters.MaxWaiting = 300
Parameters.MaxDelay = 600
Parameters.num_time_windows = 120*24

In [5]:
add_vehicles_from_data(3600*0+30, Demand)

In [6]:
def simulation_ridesharing(time_start, demands):
    # 设置开始和结束时间
    time_now = time_start + Parameters.TimeWindow
    Parameters.TimeEnd = time_now + Parameters.TimeWindow * Parameters.num_time_windows

    rounds = 1
    while (time_now < Parameters.TimeEnd + math.inf):
        print(rounds)
        if time_now < Parameters.TimeEnd:
            demands = add_requests_from_data((time_now - Parameters.TimeWindow, time_now), demands)
            #demands = add_requests_from_data2((time_now - Parameters.TimeWindow, time_now), demands)
        print(len(Parameters.Requests))

        if len(Parameters.Requests) == 0:
            finished = True
            for r in Parameters.RequestDone:
                if r.ignored:
                    continue
                else:
                    if r.DoTime == None:
                        finished = False
                        break
            if finished:
                break

        rr = combine_rr(time_now)

        compute_RV(time_now)
        Parameters.fea_set_last_iteration = Parameters.fea_set
        Parameters.fea_set = {}

        compute_RTV(rr)
        opt_assignment(time_now)

        time_now = time_now + Parameters.TimeWindow
        rounds += 1
        Parameters.current_request_id = Parameters.RequestId
        

In [7]:
# 12:03
simulation_ridesharing(3600*0,Demand)

1
115
Academic license - for non-commercial use only
rb_requests 0
idling_vehicles 2921
2
143
rb_requests 0
idling_vehicles 2845
3
160
rb_requests 0
idling_vehicles 2768
4
178
rb_requests 0
idling_vehicles 2697
5
169
rb_requests 0
idling_vehicles 2631
6
171


KeyboardInterrupt: 

In [6]:
# 结果保存
result = Parameters.RequestDone
d1 = open("./trainingdata2/result0108","wb")
pickle.dump(result,d1)
d1.close()

In [7]:
v_result = Parameters.Vehicles
v1 = open("./trainingdata2/v_result0108","wb")
pickle.dump(v_result,v1)
v1.close()

In [12]:
count = 0
for r in Parameters.RequestDone:
    if r.ignored is True:
        
        count += 1
print(count)
print(Parameters.PU)
print(len(Parameters.RequestDone))
print(Parameters.PU)

46
9912
9958
9912


In [13]:
count6 = 0
count7 = 0
count8 = 0
for v in Parameters.Vehicles:
    count6 += v.TravelTime    
    count7 += v.RbTotalTime
    count8 += v.IdleTotalTime
    
print(count6-count8)

print(count7)
print(count8)

4211820.394666671
9189.6385
5772938.186499995


In [10]:
r_sum = 0
r_num = 0
for r in Parameters.RequestDone:
    if r.ignored is False and r.shared is True:
        sharedis = r.share_distance[1]+r.share_distance[2]
        
        dis = Parameters.DistanceDict[r.PuHub][r.DoHub]
        ratio = (sharedis - dis)/dis
        r_sum += ratio
        r_num += 1

print(r_sum/r_num)

0.12339891939834839


In [11]:
r_sum0 = 0
r_num0 = 0
r_sum1 = 0
r_num1 = 0
r_sum2 = 0
r_num2 = 0
r_sum3 = 0
r_num3 = 0
r_sum4 = 0
r_num4 = 0
r_sum5 = 0
r_num5 = 0
r_sum6 = 0
r_num6 = 0
r_sum7 = 0
r_num7 = 0
r_num8 = 0
r_sum8 = 0
x = []
y = []


for r in Parameters.RequestDone:
    if r.ignored is False and r.shared is True:
        distance = Parameters.DistanceDict[r.PuHub][r.DoHub]
        dis = r.share_distance[1]+r.share_distance[2]
        ratio = dis/distance
        y.append(ratio)
        x.append(dis)
        if dis<2000:
            r_sum0 += ratio
            r_num0 += 1
        elif dis<4000:
            r_sum1 += ratio
            r_num1 += 1
        elif dis<6000:
            r_sum2 += ratio
            r_num2 += 1
        elif dis<8000:
            r_sum3 += ratio
            r_num3 += 1
        elif dis<10000:
            r_sum4 += ratio
            r_num4 += 1
        elif dis<12000:
            r_sum5 += ratio
            r_num5 += 1
        elif dis<14000:
            r_sum6 += ratio
            r_num6 += 1
        elif dis<16000:
            r_sum7 += ratio
            r_num7 += 1
        else:
            r_sum8 += ratio
            r_num8 += 1
        
print(r_sum0/r_num0, r_sum1/r_num1, r_sum2/r_num2, r_sum3/r_num3, r_sum4/r_num4, r_sum5/r_num5, r_sum6/r_num6, r_sum7/r_num7, r_sum8/r_num8)





1.071810514877333 1.1784385119626817 1.144974383382456 1.1014414461327053 1.0863856530950873 1.0724876715345326 1.064741028466603 1.0459540612799456 1.06311008190693


In [12]:
r_sum0 = 0
r_num0 = 0
r_sum1 = 0
r_num1 = 0
r_sum2 = 0
r_num2 = 0
r_sum3 = 0
r_num3 = 0
r_sum4 = 0
r_num4 = 0
r_sum5 = 0
r_num5 = 0
r_sum6 = 0
r_num6 = 0
r_sum7 = 0
r_num7 = 0
r_num8 = 0
r_sum8 = 0
x = []
y = []


for r in Parameters.RequestDone:
    if r.ignored is False and r.shared is True:
        sharedis = r.share_distance[2]
        dis = r.share_distance[1]+r.share_distance[2]
        ratio = sharedis/dis
        y.append(ratio)
        x.append(dis)
        if dis<2000:
            r_sum0 += ratio
            r_num0 += 1
        elif dis<4000:
            r_sum1 += ratio
            r_num1 += 1
        elif dis<6000:
            r_sum2 += ratio
            r_num2 += 1
        elif dis<8000:
            r_sum3 += ratio
            r_num3 += 1
        elif dis<10000:
            r_sum4 += ratio
            r_num4 += 1
        elif dis<12000:
            r_sum5 += ratio
            r_num5 += 1
        elif dis<14000:
            r_sum6 += ratio
            r_num6 += 1
        elif dis<16000:
            r_sum7 += ratio
            r_num7 += 1
        else:
            r_sum8 += ratio
            r_num8 += 1
        
print(r_sum0/r_num0, r_sum1/r_num1, r_sum2/r_num2, r_sum3/r_num3, r_sum4/r_num4, r_sum5/r_num5, r_sum6/r_num6, r_sum7/r_num7, r_sum8/r_num8)




0.7957025108158247 0.6929260085737816 0.6898167344724592 0.7150258186868671 0.7174729679738443 0.7406126538111785 0.7175406641509942 0.78268176554482 0.7158636117912416


In [13]:
sum = 0
num = 0
for r in Parameters.RequestDone:
    if r.ignored is False:
        sum += (r.DoTime-r.EarliestDoTime)
        num += 1
print(sum/num)

184.28071903024232


In [14]:
sum = 0
num = 0
for r in Parameters.RequestDone:
    if r.ignored is False:
        sum += (r.PuTime-r.RequestTime)
        num += 1
print(sum/num)

133.81337038594089
